# Comparison of spectrum between eNATL60 outputs and Altika Satellite for Region 1 and ASO months

Requisites :
 - notebook process-spectrum-comparison-eNATL60-region1-Altkia-ASO has successfully run and produced all the necessary result_*.nc in results_Altika-eNATL60-Region1-ASO
 - gonzag_cloud
 - climporn
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud'
sys.path.append(GONZAG_DIR)
import gonzag as gz


In [3]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

from scipy.signal import detrend, tukey

%matplotlib inline


In [4]:
CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


### Params

For the model

In [5]:
model = 'eNATL60'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'sossheig_bl'


For altimetry data

In [6]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'


In [13]:
=ls fresults)

../results/results_Altika/result_gonzag_netcdf_nocloud.nc


### Data

In [40]:
fresults='../results/results_Altika-eNATL60-Region1-fma/result_gonzag_netcdf_nocloud.nc'
dsn=xr.open_dataset(fresults,decode_time=False)

TypeError: open_dataset() got an unexpected keyword argument 'decode_time'

In [15]:
#clean up some remaining Nans

ds=dsn.where(nmp.isnan(dsn[name_ssh_sat])==0, drop=True)

In [39]:
ds.time

<xarray.DataArray 'time' (time: 10198)>
array(['2014-02-02T23:14:13.000000000', '2014-02-02T23:14:13.980000000',
       '2014-02-02T23:14:14.960000000', ..., '2014-04-29T23:14:02.683653120',
       '2014-04-29T23:14:03.663652864', '2014-04-29T23:14:04.643652864'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2014-02-02T23:14:13 ... 2014-04-29T23:14:0...

### Spectrum

In [38]:
%%time

ISeg_beg, ISeg_end = gz.FindUnbrokenSegments( ds.time.values, ds.distance.values, ds[name_ssh_mod].values,\
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
NbSeg, Nsl, IDEDSeg = gz.SegmentSelection(ISeg_beg, ISeg_end, np_valid_seg=70)



TypeError: The DType <class 'numpy.dtype[timedelta64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[timedelta64]'>, <class 'numpy.dtype[float64]'>)

In [ ]:
IDEDSeg[15]

In [ ]:
IDEDSeg2=nmp.delete(IDEDSeg, 15, axis=0)
NbSeg2=NbSeg-1

In [ ]:
IDEDSeg2

In [ ]:
XPs, XPm, rdist_sample = gz.Process4FFT( IDEDSeg2, ds.distance, ds[name_ssh_mod], ds[name_ssh_sat] )
Kwn, PwSpc_s, PwSpc_m = gz.ApplyFFT( IDEDSeg2, XPs, XPm, rdist_sample )

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod = nmp.mean(PwSpc_m[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s[:,:],axis=0)


In [ ]:
# Blabla for the plot:
clr_sat = '#AD0000'
clr_mod = '#008ab8'

cinfrm = str(NbSeg)+' segments\n'+str(Nsl)+' points/segment\n'+r'$\Delta$d sat.: '+str(round(rdist_sample,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn, vps_mod, clab1=name_mod+' ("'+name_ssh_mod+'")', clr1=clr_mod, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4)

plt.savefig('../plots/spectrum_SSH_'+name_sat+'-'+name_mod+'_result-from-gonzag-netcdf.png')
